In [ ]:
import sys
sys.path.append('/lustrehome/emanueleamato/.local/lib/python3.11/site-packages')
sys.path.append('/usr/local/lib/python3.8/dist-packages/torch/__init__.py')

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision.models as models

train_ds = torchvision.datasets.ImageFolder('/lustrehome/emanueleamato/ViT_Test/Train', transform=ToTensor())
test_ds = torchvision.datasets.ImageFolder('/lustrehome/emanueleamato/ViT_Test/Test', transform=ToTensor())

print("Number of train samples: ", len(train_ds))
print("Number of test samples: ", len(test_ds))
print("Detected Classes are: ", train_ds.class_to_idx)

In [ ]:
 Definisci il modello (ResNet-50)
model = models.resnet50(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(train_ds.classes))  # Cambia l'ultimo layer per adattarsi al numero di classi del tuo dataset

# Iperparametri
batch_size = 32
learning_rate = 0.01
num_epochs = 5

# Definisci la funzione di perdita e l'ottimizzatore
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Trasformazioni per la tua immagine (adatta a ciò che è necessario per il tuo dataset)
transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),
])

train_ds = torchvision.datasets.ImageFolder('/lustrehome/emanueleamato/ViT_Test/Train', transform=transform)
test_ds = torchvision.datasets.ImageFolder('/lustrehome/emanueleamato/ViT_Test/Test', transform=transform)

# Crea un iteratore per il tuo dataset
train_loader = DataLoader(dataset=train_ds, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_ds, batch_size=batch_size, shuffle=False)

# Addestra il modello
for epoch in range(num_epochs):
    print('hi')
    model.train()
    running_loss = 0.0
    total_correct = 0
    total_samples = 0
    for batch_idx, (images, labels) in enumerate(train_loader):
        # Azzera i gradienti dei parametri
        optimizer.zero_grad()
        # Passa le immagini attraverso il modello
        outputs = model(images)
        # Calcola la perdita
        loss = criterion(outputs, labels)
        # Esegui la retropropagazione del gradiente
        loss.backward()
        # Aggiorna i pesi
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        
        # Calcola l'accuratezza per batch
        _, predicted = torch.max(outputs, 1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)
        batch_accuracy = 100 * total_correct / total_samples
       
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}, Accuracy: {batch_accuracy:.2f}%')

    # Calcola la perdita media per epoca
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
